# League of Legends Project

## Part 1: Data Collection
### Ranked match data will be collected from each Summoner Name provided. 

   #### Part 1A: Data Collection from Data Dragon
   - create mapping dictionaries from the json files provided


   #### Part 1B: Data Collection from Riot API
   - extract data from RIOT API
        - the data that will be extracted from RIOT API will be:
            - fifty most recent matches from NA Challenger Players (top 300 players in North American)
            - data will vary depending on the changes in the ranking depending on the gains and losses of lp from matches
            - the data extracted was ran on 7/06/2021 at 04:30 PM
        
## Part 2: Data Analysis & Machine Learning
### Exploratory Data Analysis will be done on the match data to analyze basic trends a develop a logistic regression model to predict win/loss.
    
   #### Part 2A: Data Wrangling and Cleaning
   - wrangle and manipulate data table for consistency
   - prepare data for visualization and machine learning


   #### Part 2B: Data Visualization
   - using visualization provide insight to the questions below:
        - Who is the most played champion by lane?
        - Who are the top 3 champions that hold the highest win rate by lane?
        - What is the K/D/A for the top 10 played champions?
        - How correlated is offensive/defensive statistics to winning?
   - the visualizations will be created with Plotly and Bokeh
            
            
   #### Part 2C: Machine Learning
   - clean/wrangle/impute the data so that:
        - there are no highly correlated indepedent variables in the logistic regression model to reduce mulitcollinearity
        - create dummy variables for categorical variables to be considered in the logistic regression model
        - imput missing values with estimations for accurate predictions
   - create logistic regression model with train/test set and deteremine the accuracy of the model
            

# Part 1A

In [1]:
# league_env\Scripts\activate
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
import numpy as np
import requests
import regex as re
import json
import pprint
import time
from bs4 import BeautifulSoup

## Champion JSON

In [2]:
def create_champion_mapping_dict(file_name):
    champion_key_dict = {}
    with open(f'json_files/{file_name}', encoding = 'utf-8') as f:
        champion_json = json.load(f)
        for champion, info in champion_json['data'].items():
            champion_name = champion
            for c_key, c_value in info.items():
                if c_key == 'key':
                    champion_key_dict[champion_name] = int(c_value)
    return champion_key_dict

champion_key_dict = create_champion_mapping_dict('champion.json')

## Item JSON

In [3]:
def create_item_mapping_dict(file_name):
    item_key_dict = {}
    with open(f'json_files/{file_name}', encoding = 'utf-8') as f:
        item_data = json.load(f)
        for item, info in item_data['data'].items():
            item_id = item
            for i_key, i_value in info.items():
                if i_key == 'name':
                    item_key_dict[int(item_id)] = i_value
    return item_key_dict

item_key_dict = create_item_mapping_dict('item.json')



## Queue JSON

In [4]:
def create_queue_mapping_dict(file_name):
    queue_info_dict = {}
    with open(f'json_files/{file_name}', encoding = 'utf-8') as f:
        queue_data = json.load(f)
        for queue in queue_data:
            queue_key = queue['queueId']
            queue_name = queue['description']
            queue_notes = str(queue['notes'])
            if 'deprecated' not in queue_notes.lower():
                queue_info_dict[queue_key] = str(queue_name).upper()

    queue_info_dict_reverse = {str(v).upper(): k for (k,v) in queue_info_dict.items()}
    return queue_info_dict, queue_info_dict_reverse

queue_key_dict, queue_key_dict_reverse = create_queue_mapping_dict('queue.json')

## Summoner Spell JSON

In [5]:
def create_summoner_spell_mapping_dict(file_name):
    ss_info_dict = {}
    with open(f'json_files/{file_name}', encoding = 'utf-8') as f:
        ss_data = json.load(f)
        for s_spell, info in ss_data['data'].items():
            for key in info:
                ss_key = info['key']
                ss_name = info['name']
                ss_description = info['description']
                ss_info_dict[ss_name] = (ss_key, ss_description)
    return ss_info_dict

ss_key_dict = create_summoner_spell_mapping_dict('summoner.json')

## Season JSON

In [6]:
def create_season_mapping_dict(file_name):
    season_info_dict = {}
    with open(f"json_files/{file_name}", encoding = "utf-8") as f:
        season_data = json.load(f)
        for season in season_data:
            season_info_dict[season['id']] = season['season']
    return season_info_dict
    
season_key_dict = create_season_mapping_dict('season.json')

# Part 1B

In [7]:
REQUEST_HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.66 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Charset': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Origin': 'https://developer.riotgames.com',
    'X-Riot-Token': ''
    }

# Insert API KEY
RIOT_API = ''

## Getting PUU ID Using Summoner Name

In [8]:
def get_puu_id(summoner_name, riot_api_key = RIOT_API, request_headers = REQUEST_HEADERS, access = True):
    summoner_name_updated = summoner_name.replace(' ', '%20')
    url = 'https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/'
    account_url = url + summoner_name_updated
    riot_headers = request_headers
    riot_headers["X-Riot-Token"] = riot_api_key
    while access:
        req = requests.get(account_url, headers = riot_headers)
        if req.status_code == 200:
            account_info = json.loads(req.content.decode("utf-8"))
            puu_id = account_info['puuid']
            access = False

        elif req.status_code == 404:
            puu_id = None
            access = False

        else:
            print('Waiting for 10 seconds to refresh restriction request for getting PUU ID')
            time.sleep(10)
    return  puu_id

## Getting Match ID Using PUU ID

In [9]:
def get_match_id(puu_id, start, count, riot_api_key = RIOT_API, request_headers = REQUEST_HEADERS, access = True):
    match_id = None
    match_history_url = f'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puu_id}/ids?start={start}&count={count}'
    riot_headers = request_headers
    riot_headers["X-Riot-Token"] = riot_api_key

    while access:
        req = requests.get(match_history_url, headers = riot_headers)
        if req.status_code == 200:
            match_id_listed = json.loads(req.content.decode("utf-8"))
            match_id = ''.join(match_id_listed)
            access = False
            
        else:
            print('Waiting for 10 seconds to refresh restriction request for getting Match ID')
            time.sleep(10)
            
    return match_id

## Get Match Information Using Match ID

In [10]:
def get_match_details(match_id, riot_api_key = RIOT_API, request_headers = REQUEST_HEADERS, access = True):
    match_url = 'https://americas.api.riotgames.com/lol/match/v5/matches/' + str(match_id)
    riot_headers = request_headers
    riot_headers["X-Riot-Token"] = riot_api_key

    while access:
        req = requests.get(match_url, headers = riot_headers)
        if req.status_code == 200:
            match_details = json.loads(req.content.decode("utf-8"))
            access = False

        elif req.status_code == 404:
            match_details = None
            access = False
            
        else:
            print(req.status_code)
            print('Waiting for 10 seconds to refresh restriction request for getting Match Details')
            time.sleep(10)

    return match_details

## Cleaning and Reducing Match Data

In [11]:
def clean_and_reduce_match_data(match_details):
    match_cleaned_dict = {}
    
    match_id = None
    match_timestamp = None
    queue_id = None
    participant_details = None
    
    
    for key, value in match_details['info'].items():
        if key == 'gameId':
            match_id = value
            
        if key == 'queueId':
            queue_id = value
            
        if key == 'gameStartTimestamp':
            match_timestamp = value
            
        if key == 'participants':
            participant_details = value
    
    if queue_id == 420:
        match_cleaned_dict[(match_id, queue_id, match_timestamp)] = participant_details
                
        return match_cleaned_dict
    
    else:
        return None

In [12]:
def filter_match_data_by_puuid(match_cleaned_dict, puu_id):
    for match_id, match_details in match_cleaned_dict.items():
        for participant_dict in match_details:
            if participant_dict['puuid'] == puu_id:
                return {str(match_id[0]) + '_' + str(match_id[1]) + '_' + str(match_id[2]): participant_dict}

In [13]:
def remove_perks_attribute(match):
    for key, info in match.items():
        for key in list(info.keys()):
            if key == 'perks':
                del info[key]

    return match

## Turn Match Data into DataFrame

In [14]:
def make_match_df(match):
    match_df = pd.DataFrame.from_dict(match, orient = 'index')
    return match_df

## Create Super Function for Data Collection

In [15]:
def get_match_detail_by_summoner_name(summoner_name, beg, end, riot_api = RIOT_API, request_headers = REQUEST_HEADERS):
    
    puu_id = get_puu_id(summoner_name, riot_api, request_headers)
    match_detail_df = None
    
    if puu_id != None:
        match_id = get_match_id(puu_id, start = beg, count = end)
        match_detail = get_match_details(match_id)
        if match_detail != None:
            match_cleaned = clean_and_reduce_match_data(match_detail)
            if match_cleaned != None:
                match_cleaned_filtered = filter_match_data_by_puuid(match_cleaned, puu_id)

                match_final = remove_perks_attribute(match_cleaned_filtered)

                match_detail_df = make_match_df(match_final)
                    
                return match_detail_df
        
            else:
                print(f'MATCH ID {match_id} IS NOT A RANKED SOLO GAME')
                return 'NO MATCH ID'
    
        else:
            print(f'NO MATCH DETAIL FOUND FOR MATCH ID: {match_id}')   
            return 'NO MATCH DETAIL'
    else:
        print(f'NO PUU ID FOUND FOR SUMMONER: {summoner_name}')
        return 'NO PUU ID'

# Part 2A

## Create Super Function for Initial Data Cleanup


In [16]:
def clean_columns(data):
    data = data.reset_index()

    cleaned_cols = list(data.columns.values)[1:]

    data[['matchId', 'queueId','matchStartTimeStamp']] = data['index'].str.split('_', expand = True)

    cleaned_cols = ['matchId', 'queueId','matchStartTimeStamp'] + cleaned_cols
    data = data[cleaned_cols]

    return data

def mapping_data_values(data, mapping_key):
    if mapping_key == 'Q':
        data['queueId'] = data['queueId'].astype(int)
        data['queueId'] = data['queueId'].map(queue_key_dict)

    if mapping_key == 'I':
        item_columns = ["item0", "item1", "item2", "item3", "item4", "item5", "item6"]
        for col in item_columns:
            data[col] = data[col].map(item_key_dict)

    if mapping_key == 'S':
        summoner_columns = ['summoner1Id', 'summoner2Id']
        for col in summoner_columns:
            data[col] = data[col].map(ss_key_dict)
    
    if mapping_key == 'T':
        team_color = data['teamId'].map({100: 'Blue', 200: 'Red'})
        data.insert(1, 'teamColor', team_color)
    
    return data

In [17]:
def initial_cleanup(data):
    data = clean_columns(data)
    
    for letter in ['Q', 'I', 'S', 'T']:
        data = mapping_data_values(data, letter)        

    return data

## Combine All Super Functions

In [18]:
def get_clean_match_data_by_summoner(summoner_name, beg, end):

    df = get_match_detail_by_summoner_name(summoner_name, beg, end)
    
    if isinstance(df, pd.DataFrame):
        df = initial_cleanup(df)

    return df

In [19]:
def get_summoner_match_clean_df(summoner_name, beg, end, num_games):

    solo_game_count = 0
    df_listed = []

    print(f'STARTED PROCESSING FOR SUMMONER: {summoner_name}')
    print('')

    while solo_game_count < num_games:
        print(f'ROW {beg} HAS BEEN PROCESSED')
        df = get_clean_match_data_by_summoner(summoner_name, beg, end)
        beg += 1

        if isinstance(df, pd.DataFrame):
            solo_game_count += df.shape[0]
            print(f'SOLO GAME ROW COUNT: {solo_game_count}')
            print('')
            df_listed.append(df)
        
        elif df == 'NO MATCH ID':
            continue
        
        elif df == 'NO MATCH DETAIL':
            continue
        else:
            print('MOVING ON TO NEXT SUMMONER')
            print('')
            break
        
    if df_listed != []:
        final = pd.concat(df_listed)
        return final
    else:
        return None

## Getting Most Recent 30 Solo Ranked Matches From Challenger Level Players

In [20]:
def get_top_players(page_num):
    chall_url = f'https://na.op.gg/ranking/ladder/page={page_num}'
    response = requests.get(chall_url)
    html_soup  = BeautifulSoup(response.text, 'html.parser')
    challlenger_snames = []
    # to account for the top 5
    chall_top_5 = html_soup.find_all('li', class_ = 'ranking-highest__item')
    for challenger in chall_top_5:
        chall_sname = challenger.find('a', class_ = 'ranking-highest__name').text
        challlenger_snames.append(chall_sname)
    
    chall_rest = html_soup.find_all('tr', class_ = 'ranking-table__row')
    for challenger in chall_rest:
        challlenger_snames.append(challenger.span.text)

    return challlenger_snames

In [21]:
top_300_players = []

for page in range(1, 4):
    snames = get_top_players(page)
    top_300_players += snames

In [22]:
len(top_300_players)

300

In [23]:
top_300_players

['From Iron',
 'TSMFTX SwordArt7',
 'TL Armao',
 'Tactical',
 'IMT Destiny',
 'GeneralSn1per',
 'Josedeodo5',
 'jojopyun 16',
 'Lo conseguire',
 'TheRealPhilip',
 'Devicé',
 'Breezyyy',
 'TSMFTX Lost',
 'Josedeodo123',
 'karasuno setter',
 'PATH TO TOP',
 'no invade pls',
 'FishIord',
 'EG Impact',
 'agorinas',
 'Djalal',
 'AK Pancake',
 'Big Popsicle',
 'Neøø',
 'stayawayz',
 'Busio3',
 'MaIthaeI',
 'Biofrost ',
 'iloveyou1234',
 'Draven696969',
 'Spirax',
 'Crystal Yu',
 'RHINOOOOOOOOOOOO',
 'C9 Lebron',
 'TL Jenkins',
 'Sketchdreams',
 'TL Yeon',
 'Bvoy1',
 'Dark Wingdom',
 'Eyla',
 'Erlkönig',
 'Shadyy',
 'Dhokla',
 'SugiKim',
 'My Dream BTS',
 'concept of xdddd',
 'RAHULK1',
 'HappyBeePlayer',
 'Faisalll',
 'winstxn',
 'd1dayewanbuliao',
 'TSM FTX Donbray',
 'FZ3R0',
 'Sheiden2',
 'Karasmai v9 ',
 'Ablazeolive',
 'kingamazin',
 'Hoai Tu',
 'TSM FTX Cody Sun',
 'Good Boi',
 '804 am',
 'bradleyyy',
 'MASSGAINER44',
 'ashash3w2q23ywq',
 'Zyko',
 'Copý',
 'D0no dosko',
 'Shiba lnu',
 

In [24]:
match_df_all = []
for challenger in top_300_players:
    match_df = get_summoner_match_clean_df(challenger, 0, 1, 50)
    if isinstance(match_df, pd.DataFrame):
        match_df_all.append(match_df)

STARTED PROCESSING FOR SUMMONER: From Iron

ROW 0 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 1

ROW 1 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 2

ROW 2 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 3

ROW 3 HAS BEEN PROCESSED
MATCH ID NA1_3966281983 IS NOT A RANKED SOLO GAME
ROW 4 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 4

ROW 5 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 5

ROW 6 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 6

ROW 7 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 7

ROW 8 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 8

ROW 9 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 9

ROW 10 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 10

ROW 11 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 11

ROW 12 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 12

ROW 13 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 13

ROW 14 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 14

ROW 15 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 15

ROW 16 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 16

ROW 17 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 17

ROW 18 HAS BEEN PROCESSED
SOLO G

SOLO GAME ROW COUNT: 36

ROW 36 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 37

ROW 37 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 38

ROW 38 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 39

ROW 39 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 40

ROW 40 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 41

ROW 41 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 42

ROW 42 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 43

ROW 43 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 44

ROW 44 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 45

ROW 45 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 46

ROW 46 HAS BEEN PROCESSED
Waiting for 10 seconds to refresh restriction request for getting Match ID
Waiting for 10 seconds to refresh restriction request for getting Match ID
Waiting for 10 seconds to refresh restriction request for getting Match ID
Waiting for 10 seconds to refresh restriction request for getting Match ID
Waiting for 10 seconds to refresh restriction request for getting Match ID
Waiting for 10 seconds to refresh restriction request for gettin

Waiting for 10 seconds to refresh restriction request for getting Match ID
Waiting for 10 seconds to refresh restriction request for getting Match ID
Waiting for 10 seconds to refresh restriction request for getting Match ID
Waiting for 10 seconds to refresh restriction request for getting Match ID
Waiting for 10 seconds to refresh restriction request for getting Match ID
Waiting for 10 seconds to refresh restriction request for getting Match ID
Waiting for 10 seconds to refresh restriction request for getting Match ID
MATCH ID NA1_3938858168 IS NOT A RANKED SOLO GAME
ROW 97 HAS BEEN PROCESSED
MATCH ID NA1_3938845168 IS NOT A RANKED SOLO GAME
ROW 98 HAS BEEN PROCESSED
MATCH ID NA1_3938881951 IS NOT A RANKED SOLO GAME
ROW 99 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 32

ROW 100 HAS BEEN PROCESSED
MATCH ID NA1_3937741363 IS NOT A RANKED SOLO GAME
ROW 101 HAS BEEN PROCESSED
MATCH ID NA1_3937740124 IS NOT A RANKED SOLO GAME
ROW 102 HAS BEEN PROCESSED
MATCH ID NA1_3937716997 IS NOT A RANKED S

SOLO GAME ROW COUNT: 47

ROW 59 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 48

ROW 60 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 49

ROW 61 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 50

STARTED PROCESSING FOR SUMMONER: GeneralSn1per

ROW 0 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 1

ROW 1 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 2

ROW 2 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 3

ROW 3 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 4

ROW 4 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 5

ROW 5 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 6

ROW 6 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 7

ROW 7 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 8

ROW 8 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 9

ROW 9 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 10

ROW 10 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 11

ROW 11 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 12

ROW 12 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 13

ROW 13 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 14

ROW 14 HAS BEEN PROCESSED
MATCH ID NA1_3948849856 IS NOT A RANKED SOLO GAME
ROW

SOLO GAME ROW COUNT: 25

ROW 25 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 26

ROW 26 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 27

ROW 27 HAS BEEN PROCESSED
Waiting for 10 seconds to refresh restriction request for getting Match ID
Waiting for 10 seconds to refresh restriction request for getting Match ID
Waiting for 10 seconds to refresh restriction request for getting Match ID
Waiting for 10 seconds to refresh restriction request for getting Match ID
Waiting for 10 seconds to refresh restriction request for getting Match ID
Waiting for 10 seconds to refresh restriction request for getting Match ID
Waiting for 10 seconds to refresh restriction request for getting Match ID
SOLO GAME ROW COUNT: 28

ROW 28 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 29

ROW 29 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 30

ROW 30 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 31

ROW 31 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 32

ROW 32 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 33

ROW 33 HAS BEEN PROCESSED
SOLO GAME ROW CO

...

SOLO GAME ROW COUNT: 19

ROW 19 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 20

ROW 20 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 21

ROW 21 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 22

ROW 22 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 23

ROW 23 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 24

ROW 24 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 25

ROW 25 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 26

ROW 26 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 27

ROW 27 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 28

ROW 28 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 29

ROW 29 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 30

ROW 30 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 31

ROW 31 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 32

ROW 32 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 33

ROW 33 HAS BEEN PROCESSED
MATCH ID NA1_3961572930 IS NOT A RANKED SOLO GAME
ROW 34 HAS BEEN PROCESSED
MATCH ID NA1_3961571641 IS NOT A RANKED SOLO GAME
ROW 35 HAS BEEN PROCESSED
MATCH ID NA1_3961540510 IS NOT A RANKED SOLO GAME
ROW 36 HAS BEEN PROCESSED
MATCH I

SOLO GAME ROW COUNT: 41

ROW 43 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 42

ROW 44 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 43

ROW 45 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 44

ROW 46 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 45

ROW 47 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 46

ROW 48 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 47

ROW 49 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 48

ROW 50 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 49

ROW 51 HAS BEEN PROCESSED
429
Waiting for 10 seconds to refresh restriction request for getting Match Details
429
Waiting for 10 seconds to refresh restriction request for getting Match Details
429
Waiting for 10 seconds to refresh restriction request for getting Match Details
429
Waiting for 10 seconds to refresh restriction request for getting Match Details
429
Waiting for 10 seconds to refresh restriction request for getting Match Details
429
Waiting for 10 seconds to refresh restriction request for getting Match Details
SOLO GAME ROW COUNT: 50

STARTED PROC

SOLO GAME ROW COUNT: 4

ROW 4 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 5

ROW 5 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 6

ROW 6 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 7

ROW 7 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 8

ROW 8 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 9

ROW 9 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 10

ROW 10 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 11

ROW 11 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 12

ROW 12 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 13

ROW 13 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 14

ROW 14 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 15

ROW 15 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 16

ROW 16 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 17

ROW 17 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 18

ROW 18 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 19

ROW 19 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 20

ROW 20 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 21

ROW 21 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 22

ROW 22 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 23

ROW 23 HAS BEEN PR

MATCH ID NA1_3956847122 IS NOT A RANKED SOLO GAME
ROW 67 HAS BEEN PROCESSED
MATCH ID NA1_3956884252 IS NOT A RANKED SOLO GAME
ROW 68 HAS BEEN PROCESSED
MATCH ID NA1_3956841769 IS NOT A RANKED SOLO GAME
ROW 69 HAS BEEN PROCESSED
MATCH ID NA1_3956838075 IS NOT A RANKED SOLO GAME
ROW 70 HAS BEEN PROCESSED
MATCH ID NA1_3956813388 IS NOT A RANKED SOLO GAME
ROW 71 HAS BEEN PROCESSED
MATCH ID NA1_3956770705 IS NOT A RANKED SOLO GAME
ROW 72 HAS BEEN PROCESSED
MATCH ID NA1_3956767018 IS NOT A RANKED SOLO GAME
ROW 73 HAS BEEN PROCESSED
MATCH ID NA1_3956743878 IS NOT A RANKED SOLO GAME
ROW 74 HAS BEEN PROCESSED
MATCH ID NA1_3956740207 IS NOT A RANKED SOLO GAME
ROW 75 HAS BEEN PROCESSED
MATCH ID NA1_3956676980 IS NOT A RANKED SOLO GAME
ROW 76 HAS BEEN PROCESSED
MATCH ID NA1_3956441642 IS NOT A RANKED SOLO GAME
ROW 77 HAS BEEN PROCESSED
MATCH ID NA1_3955749833 IS NOT A RANKED SOLO GAME
ROW 78 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 16

ROW 79 HAS BEEN PROCESSED
MATCH ID NA1_3955134987 IS NOT A RANK

SOLO GAME ROW COUNT: 39

ROW 166 HAS BEEN PROCESSED
MATCH ID NA1_3943367193 IS NOT A RANKED SOLO GAME
ROW 167 HAS BEEN PROCESSED
MATCH ID NA1_3943326472 IS NOT A RANKED SOLO GAME
ROW 168 HAS BEEN PROCESSED
MATCH ID NA1_3943352656 IS NOT A RANKED SOLO GAME
ROW 169 HAS BEEN PROCESSED
MATCH ID NA1_3943330556 IS NOT A RANKED SOLO GAME
ROW 170 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 40

ROW 171 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 41

ROW 172 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 42

ROW 173 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 43

ROW 174 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 44

ROW 175 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 45

ROW 176 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 46

ROW 177 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 47

ROW 178 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 48

ROW 179 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 49

ROW 180 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 50



In [25]:
final_df = pd.concat(match_df_all)
final_df.head()

,matchId,teamColor,queueId,matchStartTimeStamp,assists,baronKills,bountyLevel,champExperience,champLevel,championId,championName,championTransform,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,deaths,detectorWardsPlaced,doubleKills,dragonKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,gameEndedInEarlySurrender,gameEndedInSurrender,goldEarned,goldSpent,individualPosition,inhibitorKills,inhibitorTakedowns,inhibitorsLost,item0,item1,item2,item3,item4,item5,item6,itemsPurchased,killingSprees,kills,lane,largestCriticalStrike,largestKillingSpree,largestMultiKill,longestTimeSpentLiving,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,neutralMinionsKilled,nexusKills,nexusLost,nexusTakedowns,objectivesStolen,objectivesStolenAssists,participantId,pentaKills,physicalDamageDealt,physicalDamageDealtToChampions,physicalDamageTaken,profileIcon,puuid,quadraKills,riotIdName,riotIdTagline,role,sightWardsBoughtInGame,spell1Casts,spell2Casts,spell3Casts,spell4Casts,summoner1Casts,summoner1Id,summoner2Casts,summoner2Id,summonerId,summonerLevel,summonerName,teamEarlySurrendered,teamId,teamPosition,timeCCingOthers,timePlayed,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,3968835167,Blue,5V5 RANKED SOLO GAMES,1625637129606,5,0,1,4679,8,89,Leona,0,10,49,49,49,5548,1,4,0,0,False,False,False,False,False,False,5090,4375,UTILITY,1,1.0,0,Mercury's Treads,Kindlegem,Runesteel Spaulders,NaN,Aegis of the Legion,NaN,Oracle Lens,21,0,1,NONE,0,0,1,170,2360,1310,2076,0,0,0,1.0,0,0,5,0,4005,1788,3626,4568,6zpHc103eNNhAt8xC6Ujhak476wF7TuTm4K4vrCU2Zuu9Y...,0,,,SUPPORT,0,16,11,17,5,11,NaN,3,NaN,-wR2sAu8IN0jiLgjgGyDQsjpnyX5tSwTwZHcIE7L9aS1TbyQ,283,From Iron,False,100,UTILITY,22,1198,12471,3325,0,6201,294,243,21,46,8,5,0,6106,226,498,1,1.0,1,0,32,5,2,16,True
0,3968753725,Blue,5V5 RANKED SOLO GAMES,1625631643943,14,0,0,12385,14,111,Nautilus,0,12,129,2082,129,33529,5,9,0,0,False,False,False,False,False,False,8304,7700,UTILITY,0,0.0,0,Mercury's Treads,Locket of the Iron Solari,Bulwark of the Mountain,Control Ward,Abyssal Mask,NaN,Oracle Lens,27,0,0,BOTTOM,0,0,0,940,10255,5238,8684,0,0,0,0.0,0,0,5,0,7385,1355,11355,4568,6zpHc103eNNhAt8xC6Ujhak476wF7TuTm4K4vrCU2Zuu9Y...,0,,,SUPPORT,0,54,30,30,11,11,NaN,3,NaN,-wR2sAu8IN0jiLgjgGyDQsjpnyX5tSwTwZHcIE7L9aS1TbyQ,283,From Iron,False,100,UTILITY,48,2128,26091,6729,1498,21169,1925,0,40,198,197,1,0,8450,136,1129,0,1.0,5,0,85,10,14,35,True
0,3968720888,Red,5V5 RANKED SOLO GAMES,1625626754567,11,0,0,4883,8,412,Thresh,0,10,1963,1963,1963,3459,1,6,0,0,True,False,True,False,False,True,5379,4925,UTILITY,0,0.0,0,Refillable Potion,Locket of the Iron Solari,Runesteel Spaulders,Control Ward,Ionian Boots of Lucidity,Faerie Charm,Oracle Lens,21,0,0,NONE,0,0,0,360,5893,2505,1259,0,0,0,0.0,0,0,10,0,5729,920,3137,4568,6zpHc103eNNhAt8xC6Ujhak476wF7TuTm4K4vrCU2Zuu9Y...,0,,,SUPPORT,0,12,15,14,5,3,NaN,3,NaN,-wR2sAu8IN0jiLgjgGyDQsjpnyX5tSwTwZHcIE7L9aS1TbyQ,283,From Iron,False,200,UTILITY,17,958,15862,3576,603,4721,581,0,25,54,12,1,0,4239,150,324,0,2.0,0,0,23,7,3,13,True
0,3965875878,Red,5V5 RANKED SOLO GAMES,1625387947486,11,0,0,10756,13,89,Leona,0,10,212,378,212,15312,5,6,0,0,True,False,False,False,False,False,8048,7600,UTILITY,0,0.0,1,NaN,Locket of the Iron Solari,Pauldrons of Whiterock,Control Ward,Mercury's Treads,Ruby Crystal,Oracle Lens,24,0,1,BOTTOM,0,0,1,950,3034,1662,4598,0,0,0,0.0,0,0,10,0,6550,1513,7227,4568,6zpHc103eNNhAt8xC6Ujhak476wF7TuTm4K4vrCU2Zuu9Y...,0,,,SOLO,0,36,19,15,9,8,NaN,5,NaN,-wR2sAu8IN0jiLgjgGyDQsjpnyX5tSwTwZHcIE7L9aS1TbyQ,283,From Iron,False,200,UTILITY,25,1878,19035,4032,1119,13027,1132,305,33,44,

In [26]:
final_df.shape

(15000, 108)

In [27]:
final_df.to_csv('export_data/most_recent_50_matches_top_300_7721.csv', index = False)

## Final Cleanup to Combined Data

In [21]:
match_data_total = pd.read_csv('export_data/most_recent_50_matches_top_300_7721.csv')
match_data_total.shape

(15000, 108)

In [22]:
match_data_total.loc[match_data_total.duplicated(keep = False) == True, ].sort_values('matchId').shape

(0, 108)

In [23]:
def final_cleanup(data):
    data['matchStartTimeStamp'] = data['matchStartTimeStamp'].astype('int64')
    data['matchStartTimeStamp'] = pd.to_datetime(data['matchStartTimeStamp'], unit='ms')
    
    item_cols = ['item0', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6']
    other_cols = ['inhibitorsLost', 'nexusLost', 'turretsLost',  'damageDealtToBuildings', 'inhibitorTakedowns', 'nexusTakedowns', 'turretTakedowns']
    null_cols = ['riotIdName', 'riotIdTagline', 'summoner1Id', 'summoner2Id', 'unrealKills', 'sightWardsBoughtInGame']
    useless_cols = ['role', 'lane', 'individualPosition']
    
    data.loc[data['teamPosition'] == 'UTILITY', 'teamPosition'] = 'SUPPORT'
    data.loc[data['individualPosition'] == 'UTILITY', 'individualPosition'] = 'SUPPORT'
    data.loc[data['championName'] == 'MonkeyKing', 'championName'] = 'Wukong'
   

    data.drop_duplicates(inplace = True)

    data.loc[: , item_cols] = data[item_cols].fillna(value = 'None')
    data.loc[: , other_cols] = data[other_cols].fillna(value = 0)
    
    data.drop(columns = null_cols, inplace = True)
    data.drop(columns = useless_cols, inplace = True)
    
    
    return data

In [24]:
match_data_total = final_cleanup(match_data_total)

In [25]:
match_data_total.shape

(15000, 99)

In [26]:
match_data_total.head()

,matchId,teamColor,queueId,matchStartTimeStamp,assists,baronKills,bountyLevel,champExperience,champLevel,championId,championName,championTransform,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,deaths,detectorWardsPlaced,doubleKills,dragonKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,gameEndedInEarlySurrender,gameEndedInSurrender,goldEarned,goldSpent,inhibitorKills,inhibitorTakedowns,inhibitorsLost,item0,item1,item2,item3,item4,item5,item6,itemsPurchased,killingSprees,kills,largestCriticalStrike,largestKillingSpree,largestMultiKill,longestTimeSpentLiving,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,neutralMinionsKilled,nexusKills,nexusLost,nexusTakedowns,objectivesStolen,objectivesStolenAssists,participantId,pentaKills,physicalDamageDealt,physicalDamageDealtToChampions,physicalDamageTaken,profileIcon,puuid,quadraKills,spell1Casts,spell2Casts,spell3Casts,spell4Casts,summoner1Casts,summoner2Casts,summonerId,summonerLevel,summonerName,teamEarlySurrendered,teamId,teamPosition,timeCCingOthers,timePlayed,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretTakedowns,turretsLost,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,3968835167,Blue,5V5 RANKED SOLO GAMES,2021-07-07 05:52:09.606,5,0,1,4679,8,89,Leona,0,10,49,49,49,5548,1,4,0,0,False,False,False,False,False,False,5090,4375,1,1.0,0,Mercury's Treads,Kindlegem,Runesteel Spaulders,None,Aegis of the Legion,None,Oracle Lens,21,0,1,0,0,1,170,2360,1310,2076,0,0,0,1.0,0,0,5,0,4005,1788,3626,4568,6zpHc103eNNhAt8xC6Ujhak476wF7TuTm4K4vrCU2Zuu9Y...,0,16,11,17,5,11,3,-wR2sAu8IN0jiLgjgGyDQsjpnyX5tSwTwZHcIE7L9aS1TbyQ,283,From Iron,False,100,SUPPORT,22,1198,12471,3325,0,6201,294,243,21,46,8,5,0,6106,226,498,1,1.0,1,32,5,2,16,True
1,3968753725,Blue,5V5 RANKED SOLO GAMES,2021-07-07 04:20:43.943,14,0,0,12385,14,111,Nautilus,0,12,129,2082,129,33529,5,9,0,0,False,False,False,False,False,False,8304,7700,0,0.0,0,Mercury's Treads,Locket of the Iron Solari,Bulwark of the Mountain,Control Ward,Abyssal Mask,None,Oracle Lens,27,0,0,0,0,0,940,10255,5238,8684,0,0,0,0.0,0,0,5,0,7385,1355,11355,4568,6zpHc103eNNhAt8xC6Ujhak476wF7TuTm4K4vrCU2Zuu9Y...,0,54,30,30,11,11,3,-wR2sAu8IN0jiLgjgGyDQsjpnyX5tSwTwZHcIE7L9aS1TbyQ,283,From Iron,False,100,SUPPORT,48,2128,26091,6729,1498,21169,1925,0,40,198,197,1,0,8450,136,1129,0,1.0,5,85,10,14,35,True
2,3968720888,Red,5V5 RANKED SOLO GAMES,2021-07-07 02:59:14.567,11,0,0,4883,8,412,Thresh,0,10,1963,1963,1963,3459,1,6,0,0,True,False,True,False,False,True,5379,4925,0,0.0,0,Refillable Potion,Locket of the Iron Solari,Runesteel Spaulders,Control Ward,Ionian Boots of Lucidity,Faerie Charm,Oracle Lens,21,0,0,0,0,0,360,5893,2505,1259,0,0,0,0.0,0,0,10,0,5729,920,3137,4568,6zpHc103eNNhAt8xC6Ujhak476wF7TuTm4K4vrCU2Zuu9Y...,0,12,15,14,5,3,3,-wR2sAu8IN0jiLgjgGyDQsjpnyX5tSwTwZHcIE7L9aS1TbyQ,283,From Iron,False,200,SUPPORT,17,958,15862,3576,603,4721,581,0,25,54,12,1,0,4239,150,324,0,2.0,0,23,7,3,13,True
3,3965875878,Red,5V5 RANKED SOLO GAMES,2021-07-04 08:39:07.486,11,0,0,10756,13,89,Leona,0,10,212,378,212,15312,5,6,0,0,True,False,False,False,False,False,8048,7600,0,0.0,1,None,Locket of the Iron Solari,Pauldrons of Whiterock,Control Ward,Mercury's Treads,Ruby Crystal,Oracle Lens,24,0,1,0,0,1,950,3034,1662,4598,0,0,0,0.0,0,0,10,0,6550,1513,7227,4568,6zpHc103eNNhAt8xC6Ujhak476wF7TuTm4K4vrCU2Zuu9Y...,0,36,19,15,9,8,5,-wR2sAu8IN0jiLgjgGyDQsjpnyX5tSwTwZHcIE7L9aS1TbyQ,283,From Iron,False,200,SUPPORT,25,1878,19035,4032,1119,13027,1132,305,33,44,90,4,0,9449,856,1201,0,1.0,6,74,8,4,34,True
4,3965931105,Blue,5V5 RANKED SOLO GAMES,2021-07-04 07:59:00.241,9,0,0,6606,10,526,Rell,0,9,0,0,0,17611,4,6,0,0,False,False,False,False,False,False,6300,5775,0,0.0,2

# Part 2B

## Questions to Visualize:

### Who is the most played champion by lane?
- select the top 3 played champions by lane and display the # of games played
    
### Who are the top 3 champions that hold the highest win rate by lane?
- visualize win rates by lane and visualize win rate by champions
- to decrease bias from lack of games played, the champions that have at least average game count will only be used when looking at win rate

### What is the K/D/A for the top 10 played champions?
- after filtering the data for top 10 played champions, visualize the average K/D/A

### Which team (Blue or Red) has a higher proportion of first bloods/first towers?
- after grouping the data by team, calculate the proportion of first bloods and first towers by team
- visualize the result in a grouped bar chart

In [27]:
test = match_data_total.groupby(['teamPosition', 'championName'])['matchId'].agg('count').reset_index()
test.rename(columns = {'matchId':'matchCount'}, inplace = True)
test_sorted = test.sort_values(['teamPosition', 'matchCount'], ascending = [True, False])
test_filtered = test_sorted.loc[test_sorted['teamPosition'] != 'AFK']

top3_bottom = list(test_filtered.loc[test_filtered['teamPosition'] == 'BOTTOM', 'championName'].head(3))
top3_support = list(test_filtered.loc[test_filtered['teamPosition'] == 'SUPPORT', 'championName'].head(3))
top3_middle = list(test_filtered.loc[test_filtered['teamPosition'] == 'MIDDLE', 'championName'].head(3))
top3_jungle = list(test_filtered.loc[test_filtered['teamPosition'] == 'JUNGLE', 'championName'].head(3))
top3_top = list(test_filtered.loc[test_filtered['teamPosition'] == 'TOP', 'championName'].head(3))

bot_mask = test_sorted['teamPosition'] == 'BOTTOM'
jung_mask = test_sorted['teamPosition'] == 'JUNGLE'
mid_mask = test_sorted['teamPosition'] == 'MIDDLE'
top_mask = test_sorted['teamPosition'] == 'TOP'
supp_mask = test_sorted['teamPosition'] == 'SUPPORT'

top3_all_lanes = test.loc[
    (bot_mask & test['championName'].isin(top3_bottom)) |
    (jung_mask & test['championName'].isin(top3_jungle)) |
    (mid_mask & test['championName'].isin(top3_middle)) |
    (top_mask & test['championName'].isin(top3_top)) |
    (supp_mask & test['championName'].isin(top3_support))
]
top3_all_lanes_sorted = top3_all_lanes.sort_values(['teamPosition', 'matchCount'], ascending = [True, False])
top3_all_lanes_sorted

,teamPosition,championName,matchCount
17,BOTTOM,Ezreal,411
52,BOTTOM,Varus,235
22,BOTTOM,Kaisa,229
113,JUNGLE,XinZhao,370
62,JUNGLE,Diana,261
80,JUNGLE,LeeSin,231
192,MIDDLE,Sylas,316
119,MIDDLE,Akali,249
160,MIDDLE,Leblanc,215
261,SUPPORT,Thresh,426


In [28]:
fig = px.bar(top3_all_lanes_sorted, x = 'teamPosition', y = 'matchCount', color = 'championName', text = 'matchCount',
#              color_discrete_map = {
#                 'Ezreal': '#1E90FF',
#                 'Varus': '#0000FF',
#                 'Aphelios': '#00008B',
#                 'Kindred': '#66CDAA',
#                 'XinZhao': '#228B22',
#                 'Viego': '#556B2F',
#                 'Akali': '#D3D3D3',
#                 'Syals': 'grey',
#                 'Leblanc': '#000000',
#                 'Nautilus': '#FFD700',
#                 'Thresh': '#DAA520',
#                 'Karma': '#B8860B',
#                 'LeeSin': '#556B2F',
#                 'Jayce': '#FFB6C1',
#                 'Camille': '#483D8B'
#             },

             labels = {'teamPosition': 'Position', 'matchCount': '# of Games Played', 'championName': 'Champion'}, title = 'Top 3 Champions Played by Position')
fig.update_layout(
    plot_bgcolor = 'rgb(0,0,0,0)',
    paper_bgcolor = 'rgb(0,0,0,0)',
    width = 800,
    height = 650,
    title_x = 0.5,
    legend = dict(bgcolor = 'white', yanchor = 'top', y = 0.94, xanchor = 'right', x = 1.20, bordercolor = 'Black', borderwidth = 1),
    legend_title = dict(font = dict(size = 12)),
)
fig.update_xaxes(categoryorder = 'total ascending')

fig.layout.legend.tracegroupgap = 8

fig.update_traces(texttemplate = '%{text} Games', textposition = 'inside', insidetextanchor = 'middle', textfont = dict(color = 'white', size = 11.5))
fig.show(renderer = 'notebook_connected')

In [29]:
win_ratio_df = match_data_total.groupby(['teamPosition', 'championName']).agg({'win': ['sum', 'count']}).droplevel(0, axis = 1)
win_ratio_df['winRatio'] = round(win_ratio_df['sum'] / win_ratio_df['count'], 4)
win_ratio_df = win_ratio_df.reset_index()
win_ratio_df = win_ratio_df.loc[win_ratio_df['teamPosition'] != 'AFK', ]
win_ratio_df

,teamPosition,championName,sum,count,winRatio
5,BOTTOM,Ahri,1,2,0.5000
6,BOTTOM,Annie,1,1,1.0000
7,BOTTOM,Aphelios,104,182,0.5714
8,BOTTOM,Ashe,21,45,0.4667
9,BOTTOM,Azir,2,2,1.0000
10,BOTTOM,Brand,2,2,1.0000
11,BOTTOM,Caitlyn,26,53,0.4906
12,BOTTOM,Cassiopeia,1,2,0.5000
13,BOTTOM,Chogath,3,6,0.5000
14,BOTTOM,Corki,3,3,1.0000


In [30]:
win_ratio_df.groupby('teamPosition').mean()['count']
bottom_avg_game_count = win_ratio_df.groupby('teamPosition').mean()['count']['BOTTOM']
jung_avg_game_count = win_ratio_df.groupby('teamPosition').mean()['count']['JUNGLE']
mid_avg_game_count = win_ratio_df.groupby('teamPosition').mean()['count']['MIDDLE']
top_avg_game_count = win_ratio_df.groupby('teamPosition').mean()['count']['TOP']
supp_avg_game_count = win_ratio_df.groupby('teamPosition').mean()['count']['SUPPORT']

In [31]:
bottom_win_rates = win_ratio_df.loc[(win_ratio_df['teamPosition'] == 'BOTTOM') & (win_ratio_df['count'] >= bottom_avg_game_count), ].sort_values(by = 'winRatio', ascending = False).head(10)
jung_win_rates = win_ratio_df.loc[(win_ratio_df['teamPosition'] == 'JUNGLE') & (win_ratio_df['count'] >= jung_avg_game_count), ].sort_values(by = 'winRatio', ascending = False).head(10)
mid_win_rates = win_ratio_df.loc[(win_ratio_df['teamPosition'] == 'MIDDLE') & (win_ratio_df['count'] >= mid_avg_game_count), ].sort_values(by = 'winRatio', ascending = False).head(10)
top_win_rates = win_ratio_df.loc[(win_ratio_df['teamPosition'] == 'TOP') & (win_ratio_df['count'] >= top_avg_game_count), ].sort_values(by = 'winRatio', ascending = False).head(10)
supp_win_rates = win_ratio_df.loc[(win_ratio_df['teamPosition'] == 'SUPPORT') & (win_ratio_df['count'] >= supp_avg_game_count), ].sort_values(by = 'winRatio', ascending = False).head(10)

win_rates_top_10 = pd.concat([bottom_win_rates, jung_win_rates, mid_win_rates, top_win_rates, supp_win_rates])
win_rates_top_10['winRatio'] = round((win_rates_top_10['winRatio'] * 100), 1)


In [32]:
def graph_top10_win_rates(data, lane):
    data = data.loc[data['teamPosition'] == lane, ]
    fig = px.bar(data, x = 'championName', y = 'winRatio', text = 'winRatio', color = 'winRatio', color_continuous_scale = px.colors.sequential.Emrld,
                 labels = {'teamPosition': 'Position', 'winRatio': 'Win Percentage', 'championName': 'Champion'},
                 title = f'Top 10 Win Rates for {lane.title()} Position')
    fig.update_layout(
        plot_bgcolor = 'rgb(0,0,0,0)',
        paper_bgcolor = 'rgb(0,0,0,0)',
        width = 800,
        height = 600,
        title_x = 0.5,
        coloraxis_showscale = False,
        )
    fig.update_xaxes(categoryorder = 'total ascending')
    fig.update_yaxes(ticksuffix = '%', visible = False)
    fig.update_traces(texttemplate = '%{text}%', textposition = 'outside', insidetextanchor = 'end', 
                      textfont_color = '#000000', textfont_size = 14)

    return fig

In [33]:
team_positions = ['BOTTOM', 'SUPPORT', 'TOP', 'MIDDLE', 'JUNGLE']
top_10_by_position = []
for position in team_positions:
    fig = graph_top10_win_rates(win_rates_top_10, position)
    top_10_by_position.append(fig)

In [34]:
for fig in top_10_by_position:
    fig.show(renderer = 'notebook_connected')

In [35]:
average_kda = match_data_total.loc[:, ['championName', 'kills', 'deaths', 'assists', 'matchId']].groupby('championName').agg({
    'kills': 'mean', 'deaths': 'mean', 'assists': 'mean', 'matchId': 'count'
})
average_kda.rename(columns = {'matchId': 'game_count'}, inplace = True)
average_kda_df = average_kda.reset_index()
average_kda_df_top_10_gamecount = average_kda_df.sort_values(by = 'game_count', ascending = False).iloc[:10, :]

average_kda_df_top_10_gamecount_cleaned = average_kda_df_top_10_gamecount.rename(
    columns = {'championName': 'Champion', 'kills': 'Average Kills', 'deaths': 'Average Deaths', 'assists': 'Average Assists', 'game_count': '# of Games'}
)
average_kda_df_top_10_gamecount_cleaned['Average Kills'] = round(average_kda_df_top_10_gamecount_cleaned['Average Kills'], 1)
average_kda_df_top_10_gamecount_cleaned['Average Deaths'] = round(average_kda_df_top_10_gamecount_cleaned['Average Deaths'], 1)
average_kda_df_top_10_gamecount_cleaned['Average Assists'] = round(average_kda_df_top_10_gamecount_cleaned['Average Assists'], 1)

average_kda_df_top_10_nogamecount =  average_kda_df_top_10_gamecount_cleaned.drop(columns = '# of Games')
average_kda_df_long = pd.melt(average_kda_df_top_10_nogamecount, id_vars = ['Champion'], var_name = 'stat_type', value_name = 'stat_count')

In [36]:
fig = px.bar(average_kda_df_long, x = 'Champion', y = 'stat_count', color = 'stat_type', barmode = 'group', text = 'stat_count',
             labels = {'stat_count': '', 'stat_type': 'Stat Type'}, title = 'KDA for Top 10 Played Champions')

fig.update_layout(
    plot_bgcolor = 'rgb(0,0,0,0)',
    paper_bgcolor = 'rgb(0,0,0,0)',
    width = 950,
    height = 600,
    title_x = 0.5,
    legend = dict(bgcolor = 'white', y = 1.2, x = 0.0, bordercolor = 'Black', borderwidth = 1),
    legend_title = dict(font = dict(size = 11)),
)
fig.update_xaxes(categoryorder = 'total ascending')
fig.update_traces(texttemplate = '%{text}', textposition = 'inside', insidetextanchor = 'end', 
                      textfont_color = '#000000')
fig.show(renderer = 'notebook_connected')

# Part 2C

## Logistic Regression to Determine Win/Loss

In [37]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

## Creating Dummy Variables

In [38]:
match_data_total.select_dtypes(include='object').head()

,teamColor,queueId,championName,item0,item1,item2,item3,item4,item5,item6,puuid,summonerId,summonerName,teamPosition
0,Blue,5V5 RANKED SOLO GAMES,Leona,Mercury's Treads,Kindlegem,Runesteel Spaulders,None,Aegis of the Legion,None,Oracle Lens,6zpHc103eNNhAt8xC6Ujhak476wF7TuTm4K4vrCU2Zuu9Y...,-wR2sAu8IN0jiLgjgGyDQsjpnyX5tSwTwZHcIE7L9aS1TbyQ,From Iron,SUPPORT
1,Blue,5V5 RANKED SOLO GAMES,Nautilus,Mercury's Treads,Locket of the Iron Solari,Bulwark of the Mountain,Control Ward,Abyssal Mask,None,Oracle Lens,6zpHc103eNNhAt8xC6Ujhak476wF7TuTm4K4vrCU2Zuu9Y...,-wR2sAu8IN0jiLgjgGyDQsjpnyX5tSwTwZHcIE7L9aS1TbyQ,From Iron,SUPPORT
2,Red,5V5 RANKED SOLO GAMES,Thresh,Refillable Potion,Locket of the Iron Solari,Runesteel Spaulders,Control Ward,Ionian Boots of Lucidity,Faerie Charm,Oracle Lens,6zpHc103eNNhAt8xC6Ujhak476wF7TuTm4K4vrCU2Zuu9Y...,-wR2sAu8IN0jiLgjgGyDQsjpnyX5tSwTwZHcIE7L9aS1TbyQ,From Iron,SUPPORT
3,Red,5V5 RANKED SOLO GAMES,Leona,None,Locket of the Iron Solari,Pauldrons of Whiterock,Control Ward,Mercury's Treads,Ruby Crystal,Oracle Lens,6zpHc103eNNhAt8xC6Ujhak476wF7TuTm4K4vrCU2Zuu9Y...,-wR2sAu8IN0jiLgjgGyDQsjpnyX5tSwTwZHcIE7L9aS1TbyQ,From Iron,SUPPORT
4,Blue,5V5 RANKED SOLO GAMES,Rell,Plated Steelcaps,Refillable Potion,Bulwark of the Mountain,Control Ward,Abyssal Mask,Kindlegem,Oracle Lens,6zpHc103eNNhAt8xC6Ujhak476wF7TuTm4K4vrCU2Zuu9Y...,-wR2sAu8IN0jiLgjgGyDQsjpnyX5tSwTwZHcIE7L9aS1TbyQ,From Iron,SUPPORT


In [39]:
dummys = pd.get_dummies(match_data_total[['teamColor', 'teamPosition', 'championName', 'item0', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6']])
dummys.head()
dummys.shape

(15000, 1245)

In [40]:
match_data_total.shape

(15000, 99)

In [41]:
match_data_total_dummys = pd.concat([match_data_total, dummys], axis = 1)
match_data_total_dummys.head()

,matchId,teamColor,queueId,matchStartTimeStamp,assists,baronKills,bountyLevel,champExperience,champLevel,championId,championName,championTransform,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,deaths,detectorWardsPlaced,doubleKills,dragonKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,gameEndedInEarlySurrender,gameEndedInSurrender,goldEarned,goldSpent,inhibitorKills,inhibitorTakedowns,inhibitorsLost,item0,item1,item2,item3,item4,item5,item6,itemsPurchased,killingSprees,kills,largestCriticalStrike,largestKillingSpree,largestMultiKill,longestTimeSpentLiving,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,neutralMinionsKilled,nexusKills,nexusLost,nexusTakedowns,objectivesStolen,objectivesStolenAssists,participantId,pentaKills,physicalDamageDealt,physicalDamageDealtToChampions,physicalDamageTaken,profileIcon,puuid,quadraKills,spell1Casts,spell2Casts,spell3Casts,spell4Casts,summoner1Casts,summoner2Casts,summonerId,summonerLevel,summonerName,teamEarlySurrendered,teamId,teamPosition,timeCCingOthers,timePlayed,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretTakedowns,turretsLost,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win,teamColor_Blue,teamColor_Red,teamPosition_AFK,teamPosition_BOTTOM,teamPosition_JUNGLE,teamPosition_MIDDLE,teamPosition_SUPPORT,teamPosition_TOP,championName_Aatrox,championName_Ahri,championName_Akali,championName_Alistar,championName_Amumu,championName_Anivia,championName_Annie,championName_Aphelios,championName_Ashe,championName_AurelionSol,championName_Azir,championName_Bard,championName_Blitzcrank,championName_Brand,championName_Braum,championName_Caitlyn,championName_Camille,championName_Cassiopeia,championName_Chogath,championName_Corki,championName_Darius,championName_Diana,championName_DrMundo,championName_Draven,championName_Ekko,championName_Elise,championName_Evelynn,championName_Ezreal,championName_FiddleSticks,championName_Fiora,championName_Fizz,championName_Galio,championName_Gangplank,championName_Garen,championName_Gnar,championName_Gragas,championName_Graves,championName_Gwen,championName_Hecarim,championName_Heimerdinger,championName_Illaoi,championName_Irelia,championName_Ivern,championName_Janna,championName_JarvanIV,championName_Jax,championName_Jayce,championName_Jhin,championName_Jinx,championName_Kaisa,championName_Kalista,championName_Karma,championName_Karthus,championName_Kassadin,championName_Katarina,championName_Kayle,championName_Kayn,championName_Kennen,championName_Khazix,championName_Kindred,championName_Kled,championName_KogMaw,championName_Leblanc,championName_LeeSin,championName_Leona,championName_Lillia,championName_Lissandra,championName_Lucian,championName_Lulu,championName_Lux,championName_Malphite,championName_Malzahar,championName_Maokai,championName_MasterYi,championName_MissFortune,championName_Mordekaiser,championName_Morgana,championName_Nami,championName_Nasus,championName_Nautilus,championName_Neeko,championName_Nidalee,championName_Nocturne,championName_Nunu,championName_Olaf,championName_Orianna,championName_Ornn,championName_Pantheon,championName_Poppy,championName_Pyke,championName_Qiyana,championName_Quinn,championName_Rakan,championName_Rammus,championName_RekSai,championName_Rell,championName_Renekton,championName_Rengar,championName_Riven,championName_Rumble,championName_Ryze,championName_Samira,championName_Sejuani,championName_Senna,championName_Seraphine,championName_Sett,championName_Shaco,championName_Shen,championName_Shyvana,championName_Singed,championName_Sion,championName_Sivir,championName_Skarner,championName_Sona,championName_Soraka,championName_Swain,championName_Sylas,championName_Syndra,champio

## Checking for Mulitcollinearity
- running a correlation matrix to see if any of hte independent variables have a correlation greater than .80 or less than -.80

In [42]:
corr_matrix = match_data_total_dummys.corr().abs()
high_corr_var = np.where(corr_matrix > 0.8)
high_corr_var = [(corr_matrix.index[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]
high_corr_var

[('champExperience', 'champLevel'),
 ('champExperience', 'goldEarned'),
 ('champExperience', 'goldSpent'),
 ('champExperience', 'timePlayed'),
 ('champExperience', 'totalDamageDealt'),
 ('champLevel', 'goldEarned'),
 ('champLevel', 'goldSpent'),
 ('champLevel', 'timePlayed'),
 ('championTransform', 'championName_Kayn'),
 ('consumablesPurchased', 'visionWardsBoughtInGame'),
 ('damageDealtToBuildings', 'damageDealtToTurrets'),
 ('damageSelfMitigated', 'totalDamageTaken'),
 ('deaths', 'totalTimeSpentDead'),
 ('goldEarned', 'goldSpent'),
 ('goldEarned', 'totalDamageDealt'),
 ('goldEarned', 'totalDamageDealtToChampions'),
 ('goldSpent', 'totalDamageDealt'),
 ('goldSpent', 'totalDamageDealtToChampions'),
 ('inhibitorsLost', 'turretsLost'),
 ('killingSprees', 'kills'),
 ('kills', 'largestKillingSpree'),
 ('magicDamageDealt', 'magicDamageDealtToChampions'),
 ('magicDamageTaken', 'totalDamageTaken'),
 ('neutralMinionsKilled', 'teamPosition_JUNGLE'),
 ('participantId', 'teamId'),
 ('participantI

In [43]:
# consider variables that show at least low levels of correlations to win to be used in the model
high_corr_to_win = match_data_total_dummys.corr().abs()['win'].reset_index().rename(columns = {'index': 'independent_variables', 'win': 'correlation_to_win'}).sort_values(by = 'correlation_to_win', ascending = False).iloc[1: , :]
strong_corr_vars_df = high_corr_to_win.loc[high_corr_to_win['correlation_to_win'] >= 0.25]
strong_corr_vars = list(high_corr_to_win.loc[high_corr_to_win['correlation_to_win'] >= 0.25, 'independent_variables'])
strong_corr_vars_df

,independent_variables,correlation_to_win
78,turretsLost,0.719771
40,nexusLost,0.621261
27,inhibitorsLost,0.565520
3,bountyLevel,0.468920
13,deaths,0.443260
76,turretKills,0.375614
1,assists,0.374026
77,turretTakedowns,0.365979
32,largestKillingSpree,0.355387
70,totalTimeSpentDead,0.339411


In [44]:
multicollinearity_vars = []
for corr_pair in high_corr_var:
    if corr_pair[0] in strong_corr_vars:
        if corr_pair[1] in strong_corr_vars:
            multicollinearity_vars.append(corr_pair)
multicollinearity_vars

[('damageDealtToBuildings', 'damageDealtToTurrets'),
 ('deaths', 'totalTimeSpentDead'),
 ('inhibitorsLost', 'turretsLost'),
 ('killingSprees', 'kills'),
 ('kills', 'largestKillingSpree')]

In [45]:
vars_to_remove = ['inhibitorsLost', 'damageDealtToBuildings', 'totalTimeSpentDead', 'kills']

for var in vars_to_remove:
    if var in strong_corr_vars:
        strong_corr_vars.remove(var)
strong_corr_vars

['turretsLost',
 'nexusLost',
 'bountyLevel',
 'deaths',
 'turretKills',
 'assists',
 'turretTakedowns',
 'largestKillingSpree',
 'nexusTakedowns',
 'inhibitorTakedowns',
 'damageDealtToTurrets',
 'damageDealtToObjectives',
 'killingSprees',
 'inhibitorKills',
 'goldEarned',
 'largestMultiKill']

In [48]:
x = match_data_total.loc[:, strong_corr_vars]

y = match_data_total['win']

In [49]:
y = y.astype('int')

## Training Model

In [50]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state = 1)
# logistic_regression = LogisticRegression(solver='liblinear')
# logistic_regression.fit(x_train, y_train)
# y_pred = logistic_regression.predict(x_test)
# accuracy = metrics.accuracy_score(y_test, y_pred)
# accuracy_percentage = 100 * accuracy
# accuracy_percentage

In [51]:
C_values = list(range(1, 20))
C_scores = {}

for c in C_values:
    logistic_regression = LogisticRegression(solver='liblinear', C=c, random_state = 1)
    logistic_regression.fit(x_train, y_train)
    y_pred = logistic_regression.predict(x_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    C_scores[c] = round(accuracy, 3)
    
C_scores

{1: 0.954,
 2: 0.954,
 3: 0.954,
 4: 0.954,
 5: 0.954,
 6: 0.954,
 7: 0.954,
 8: 0.954,
 9: 0.954,
 10: 0.954,
 11: 0.954,
 12: 0.954,
 13: 0.954,
 14: 0.954,
 15: 0.954,
 16: 0.955,
 17: 0.954,
 18: 0.954,
 19: 0.954}

## Testing Model

In [52]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.30, random_state = 1)
logistic_regression = LogisticRegression(solver='liblinear', C = 16)
logistic_regression.fit(x_train, y_train)
y_pred = logistic_regression.predict(x_test)
accuracy = metrics.accuracy_score(y_test, y_pred)
accuracy_percentage = 100 * accuracy
accuracy_percentage

95.51111111111112

In [53]:
cm = metrics.confusion_matrix(y_test, y_pred)
cm

array([[1971,  118],
       [  84, 2327]], dtype=int64)

In [54]:
import plotly.figure_factory as ff
fig = ff.create_annotated_heatmap(cm,
                x = ['Predicted Loss', 'Predicted Win'],
                y= ['Actual Loss', 'Actual Win'],
                colorscale='RdBu',
                showscale = True)

fig.update_layout(title = f'Correlation Matrix - Accuracy Score: {round(accuracy_percentage)}%',
                  width = 600,
                  height = 600,
                  title_x = 0.5)
fig.show(renderer = 'notebook_connected')